<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/Jay_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downl

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("word2vec").getOrCreate()
MAX_MEMORY='4g'
spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [4]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)

# Show DataFrame
df.show()

+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+---------------+-------------+-----------------+
|beer_id|        username|      date|                text|                look|               smell|               taste|                feel|             overall|               score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|   brewery_city|brewery_state|    brewery_types|
+-------+----------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-----------

In [6]:
# Check the schema for numeric conversion
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string]>

In [7]:
df.count()

2127677

In [8]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')

In [9]:
# confirm look, smell, taste, overall, and score
df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+
| 271781|    bluejacket74|2017-03-17|   750 ml bottle,...| 4.0|  4.0|  4.0|4.25|    4.0| 4.03|Motorbreath Imper...|          OH|American Imperial...|Limited (brewed o...|10.8|     28094|Four String Brewi...|    Columbus|           OH| Brewery, Bar|
| 18

In [10]:
df.count()

1973273

In [11]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))
df = df.withColumn('smell_numeric', df['smell'].cast('float'))
df = df.withColumn('taste_numeric', df['taste'].cast('float'))
df = df.withColumn('feel_numeric', df['feel'].cast('float'))
df = df.withColumn('overall_numeric', df['overall'].cast('float'))
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [17]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [18]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [19]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [20]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
beer_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+---

In [22]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
style_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+--

In [24]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [26]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [28]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)
final_df.show()

In [31]:
# Create a df that had only the columns needed, leaving final text in this version for clustering later
compare_df = final_df.select('look_numeric', 'overall_numeric','features', 'Final_text')

In [32]:
compare_df.show()

+------------+---------------+--------------------+--------------------+
|look_numeric|overall_numeric|            features|          Final_text|
+------------+---------------+--------------------+--------------------+
|         4.0|            4.0|[motorbreathimper...|MotorbreathImperi...|
|         4.0|           3.75|[paybackpilsner, ...|PaybackPilsner Bo...|
|         4.0|            4.0|[paybackpilsner, ...|PaybackPilsner Bo...|
|        3.25|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|
|        3.75|           3.75|[paybackpilsner, ...|PaybackPilsner Bo...|
|         4.5|           3.75|[paybackpilsner, ...|PaybackPilsner Bo...|
|         3.5|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|
|         3.5|            3.5|[hellhammer, ryeb...|

In [ ]:
#
# Next 5 cells commented out because we already generated the model
#


# Create w2v model hyperparameters
#word2vec = Word2Vec(
#    vectorSize=65,
#    seed=42,
#    inputCol="features",
#    outputCol="model"
#).setMaxIter(2)

In [ ]:
# Fit the model, this takes time
#w2v_review_model = word2vec.fit(final_df)

In [ ]:
# show some model vectors
#w2v_review_model.getVectors().show()

+--------------------+--------------------+
|                word|              vector|
+--------------------+--------------------+
|               mells|[0.03909590840339...|
|             tasties|[0.09013923257589...|
|    sidebysidebyside|[0.20330214500427...|
|        morph3/10/15|[-0.0720658451318...|
|              profle|[-0.0295072738081...|
|           professed|[-0.0607991032302...|
|            macdaddy|[-0.0618058256804...|
|brewer'spaleale(n...|[-0.2465905994176...|
|               31211|[-0.1716884523630...|
|palmettohopharves...|[-0.0251995716243...|
|  experimentalj-hole|[-0.0857357680797...|
|          machfive55|[-0.1270782351493...|
|              300day|[0.03120961971580...|
|          iteresting|[-0.0693334937095...|
|          fivehopipa|[-0.2610330283641...|
|               chary|[0.00651914766058...|
|          nonfruited|[0.34797608852386...|
|       eyewateringly|[0.15294347703456...|
|100barrelseries#2...|[-0.1572935581207...|
|               motts|[0.2009695

In [ ]:
# check the model accuracy by checking synonyms 
#w2v_review_model.findSynonymsArray("light", 5)

[('medium', 0.6870060563087463),
 ('ligh', 0.6730795502662659),
 ('slight', 0.661352813243866),
 ('mild', 0.6593009829521179),
 ('mostly', 0.6529689431190491)]

In [ ]:
# write the model to disk
#w2v_review_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/review.md")

In [33]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [34]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('medium', 0.6870060563087463),
 ('ligh', 0.6730795502662659),
 ('slight', 0.661352813243866),
 ('mild', 0.6593009829521179),
 ('mostly', 0.6529689431190491)]

In [35]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(compare_df)

In [36]:
w2v_model_df.show()

+------------+---------------+--------------------+--------------------+--------------------+
|look_numeric|overall_numeric|            features|          Final_text|               model|
+------------+---------------+--------------------+--------------------+--------------------+
|         4.0|            4.0|[motorbreathimper...|MotorbreathImperi...|[-0.0663283048436...|
|         4.0|           3.75|[paybackpilsner, ...|PaybackPilsner Bo...|[0.01337589820226...|
|         4.0|            4.0|[paybackpilsner, ...|PaybackPilsner Bo...|[0.11474443169740...|
|        3.25|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|[-0.0168181593762...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|[-0.0148857976768...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|[-0.0051498340546...|
|        3.75|            3.5|[paybackpilsner, ...|PaybackPilsner Bo...|[-0.0365503356398...|
|        3.75|           3.75|[paybackpilsner, ...|PaybackPi

In [37]:
# Remove features to slim down the df (memory)
w2v_model_df = w2v_model_df.select('look_numeric','overall_numeric', 'model', 'Final_text')

In [38]:
#  Force the df into memory for faster processing
w2v_model_df.cache()
w2v_model_df.show()

+------------+---------------+--------------------+--------------------+
|look_numeric|overall_numeric|               model|          Final_text|
+------------+---------------+--------------------+--------------------+
|         4.0|            4.0|[-0.0663283048436...|MotorbreathImperi...|
|         4.0|           3.75|[0.01337589820226...|PaybackPilsner Bo...|
|         4.0|            4.0|[0.11474443169740...|PaybackPilsner Bo...|
|        3.25|            3.5|[-0.0168181593762...|PaybackPilsner Bo...|
|        3.75|            3.5|[-0.0148857976768...|PaybackPilsner Bo...|
|        3.75|            3.5|[-0.0051498340546...|PaybackPilsner Bo...|
|        3.75|            3.5|[-0.0365503356398...|PaybackPilsner Bo...|
|        3.75|           3.75|[-0.0303566464262...|PaybackPilsner Bo...|
|         4.5|           3.75|[0.00771237849795...|PaybackPilsner Bo...|
|         3.5|            3.5|[0.15409735341866...|PaybackPilsner Bo...|
|         3.5|            3.5|[0.02001750677529...|

In [39]:
# just grab what you need for linear regression
split_df = w2v_model_df.select('look_numeric', 'overall_numeric', 'model')

In [40]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = split_df.randomSplit([0.60, 0.40])

In [41]:
training.show()

+------------+---------------+--------------------+
|look_numeric|overall_numeric|               model|
+------------+---------------+--------------------+
|         1.0|            1.0|[-0.1157266438007...|
|         1.0|            1.0|[-0.1002899607022...|
|         1.0|            1.0|[-0.0729491711166...|
|         1.0|            1.0|[-0.0688415827850...|
|         1.0|            1.0|[-0.0593767903961...|
|         1.0|            1.0|[-0.0478301577580...|
|         1.0|            1.0|[-0.0423845817673...|
|         1.0|            1.0|[-0.0247685270089...|
|         1.0|            1.0|[-0.0218733304313...|
|         1.0|            1.0|[-0.0184608456865...|
|         1.0|            1.0|[-0.0161511012042...|
|         1.0|            1.0|[-0.0126860639410...|
|         1.0|            1.0|[-0.0046073434090...|
|         1.0|            1.0|[-0.0025670282685...|
|         1.0|            1.0|[-0.0018055374002...|
|         1.0|            1.0|[0.00158676968872...|
|         1.

In [42]:
# Convert the df to pandas to use sklearn
training_pandas_df = training.toPandas()

In [43]:
# Show the pandas df
training_pandas_df.head()

,look_numeric,overall_numeric,model
0,1.0,1.0,"[-0.11572664380073548, -0.21177025735378266, 0..."
1,1.0,1.0,"[-0.10028996070226033, 0.18440489346782366, 0...."
2,1.0,1.0,"[-0.07294917111662594, 0.021757342733090393, -..."
3,1.0,1.0,"[-0.06884158278505007, -0.10826455677549043, 0..."
4,1.0,1.0,"[-0.05937679039613743, 0.054797247745487505, 0..."


In [44]:
training_pandas_df.dtypes

look_numeric       float32
overall_numeric    float32
model               object
dtype: object

In [45]:
# Create the linear regression model
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [46]:
# Fit the overall rating into the model
lin_overall_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['overall_numeric'])

In [47]:
# Verify the score, very low so no strong correlation between w2v model and overall rating
lin_overall_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['overall_numeric'])

0.3371138180677927

In [48]:
# Show the coeficients
lin_overall_train_model.coef_

array([ 0.41862995, -0.81456952,  0.79084669, -0.37257978,  0.27270892,
        0.4410475 , -0.73740193,  0.14757209, -0.18132043, -1.12962795,
        0.6398336 , -0.84552988, -0.41661833,  1.09556504,  0.26520997,
       -0.11966867, -0.93179808,  0.87395745,  0.53100083,  0.1165403 ,
        0.12734666, -1.23781141, -0.99417721,  0.37100428,  0.84263026,
        1.12519331, -0.46009776,  0.06327202,  0.32361941, -1.6367692 ,
        0.71207149, -0.29438459, -0.24974573,  1.27863291, -0.27412412,
        0.80083368, -0.01344972,  0.7868559 , -0.8885303 , -0.62212273,
       -0.09495085, -0.24275105,  0.31159227, -0.03539022, -0.39608538,
        0.71405626, -0.08272329, -1.25873066,  0.36481365, -0.27854118,
        0.69678869, -0.64428112,  1.2093015 ,  0.00795049, -0.17902751,
        0.17798804,  0.35903192, -1.20038386,  0.30121142, -1.57155707,
        1.69767489,  0.26618437,  0.24966327,  0.03182962, -0.30432975])

In [49]:
# Fit the model for the look rating
lin_look_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['look_numeric'])

In [50]:
# Get the score for the look rating
lin_look_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['look_numeric'])

0.26760635002311406

In [51]:
# Show the coeficients
lin_look_train_model.coef_

array([-0.03274384, -0.58829878,  0.62779879, -0.15396385, -0.08152624,
        0.65593954, -0.45334408, -0.04295542,  0.18751141, -1.33343711,
        0.41581701, -0.46528384, -0.10960816,  1.01399272,  0.02155099,
        0.10012492, -0.47415692,  0.96009713,  0.33152538,  0.39682514,
        0.06631629, -0.18066401, -0.62501577,  0.08449315,  0.2235808 ,
        1.02715099, -0.48263594, -0.02879044, -0.01402637, -1.18124042,
        0.24231857,  0.05122062, -0.14529452,  0.67961121, -0.39733442,
        0.93215281,  0.04721972,  0.6067264 , -0.52510971,  0.20971464,
        0.34157839, -0.24414866, -0.05520494,  0.11392512, -0.05655799,
        0.63101225, -0.44502965, -0.58537169,  0.02185226, -0.00875076,
        0.54526934, -0.57530115,  0.6501575 , -0.07957997, -0.05257745,
        0.1718559 ,  0.66709535, -0.90371172, -0.08520627, -0.88017852,
        1.09198517, -0.36141005,  0.20243158, -0.39473088,  0.12881629])

In [ ]:
#
# Commented out since we are going to use sklearn and pandas (processing was to slow for spark)
#
#  even at 1% on the training data
#

# Prepare the hyperparameters
#from pyspark.ml.regression import LinearRegression, LinearRegressionModel
#lin_model = LinearRegression(
#    maxIter=5,
#    regParam=0.3,
#    featuresCol="model",
#    labelCol="look_numeric"
#)

In [ ]:
# Fit the look review to a model (takes time)
#lin_look_train_model = lin_model.fit(training)

In [ ]:
# Save the model for later use
#lin_look_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [ ]:
# Read the model
#loaded_lin_look_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_look_train.md")

In [ ]:
# Transform the training data to peek at the predictions
#lin_look_df = loaded_lin_look_train_model.transform(training)

In [ ]:
# show a sample of the predictions
#lin_look_df.select('look_numeric', 'prediction', 'Final_text').show(truncate=False)

+------------+------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Experiment
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator

#evaluator = MulticlassClassificationEvaluator(
#    labelCol="look_numeric", 
#    predictionCol="prediction", 
#    metricName="accuracy"
#)

#accuracy = evaluator.evaluate(lin_look_df)

#print("Accuracy of model at predicting reviews was: %f" % accuracy)

Accuracy of model at predicting reviews was: 0.000000


In [57]:
# Prepare the cluster dataframe
w2v_cluster = w2v_model_df.select('Final_text', 'model').withColumnRenamed("model","features").withColumnRenamed("Final_text", "text")
w2v_cluster.show()

+--------------------+--------------------+
|                text|            features|
+--------------------+--------------------+
|MotorbreathImperi...|[-0.0663283048436...|
|PaybackPilsner Bo...|[0.01337589820226...|
|PaybackPilsner Bo...|[0.11474443169740...|
|PaybackPilsner Bo...|[-0.0168181593762...|
|PaybackPilsner Bo...|[-0.0148857976768...|
|PaybackPilsner Bo...|[-0.0051498340546...|
|PaybackPilsner Bo...|[-0.0365503356398...|
|PaybackPilsner Bo...|[-0.0303566464262...|
|PaybackPilsner Bo...|[0.00771237849795...|
|PaybackPilsner Bo...|[0.15409735341866...|
|HELLHAMMER RyeBee...|[0.02001750677529...|
|HELLHAMMER RyeBee...|[0.11411257665604...|
|VanillaPorter Ame...|[-0.1063658321675...|
|VanillaPorter Ame...|[-0.1119181135597...|
|VanillaPorter Ame...|[0.10577825705210...|
|VanillaPorter Ame...|[0.00122797399759...|
|VanillaPorter Ame...|[-0.0182727483513...|
|VanillaPorter Ame...|[-0.1738284160341...|
|VanillaPorter Ame...|[-0.1093981198781...|
|VanillaPorter Ame...|[-0.106967

In [58]:
from pyspark.ml.feature import PCA
from pyspark.mllib.linalg import Vectors

pca = PCA(k=10, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(w2v_cluster)
result = model.transform(w2v_cluster)
result.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [59]:
result = result.select("text", "pcaFeatures")
w2v_cluster = result.withColumnRenamed("pcaFeatures", "features")
w2v_cluster.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [60]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

# Trains a k-means model.
kmeans = KMeans().setK(5).setSeed(1)
km_model = kmeans.fit(w2v_cluster)

In [61]:
# Make predictions
predictions = km_model.transform(w2v_cluster)

# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator(distanceMeasure="cosine")


# silhouette score goes between -1, 1. 1 is better. Negative is bad
silhouette = evaluator.evaluate(predictions)
print("Silhouette with cosine distance = " + str(silhouette))

Silhouette with cosine distance = 0.28732020430050464


In [62]:
predictions.show()

+--------------------+--------------------+----------+
|                text|            features|prediction|
+--------------------+--------------------+----------+
|MotorbreathImperi...|[-0.1226726908914...|         4|
|PaybackPilsner Bo...|[-0.0831662928690...|         2|
|PaybackPilsner Bo...|[-0.2143171111567...|         0|
|PaybackPilsner Bo...|[0.10568091120210...|         2|
|PaybackPilsner Bo...|[-0.0299112735234...|         0|
|PaybackPilsner Bo...|[-0.0918525572272...|         0|
|PaybackPilsner Bo...|[0.00578659559916...|         2|
|PaybackPilsner Bo...|[0.01458754617712...|         0|
|PaybackPilsner Bo...|[-0.0030334419817...|         0|
|PaybackPilsner Bo...|[-1.9927213411598...|         3|
|HELLHAMMER RyeBee...|[-0.0601875544963...|         0|
|HELLHAMMER RyeBee...|[-0.0630724322255...|         0|
|VanillaPorter Ame...|[-0.4864597013766...|         1|
|VanillaPorter Ame...|[-0.0061409325848...|         4|
|VanillaPorter Ame...|[-0.4102971163293...|         1|
|VanillaPo

In [64]:
predictions.filter("prediction=0").show(100, truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# Just looking at spark parameters
print(spark.sparkContext.defaultParallelism)

2


In [ ]:
# More spark params
s = spark.sparkContext._jsc.sc().getExecutorMemoryStatus().keys()
print(s)
l = str(s).replace("Set(","").replace(")","").split(", ")
print(l)

d = set()
for i in l:
    d.add(i.split(":")[0])
print(len(d))

print(spark.sparkContext._jsc.sc().getExecutorMemoryStatus().size())

Set(492190d9606c:38029)
['492190d9606c:38029']
1
1


In [52]:
# Get the spark config
spark.sparkContext._conf.getAll() 

[('spark.driver.host', 'f2e9c2339494'),
 ('spark.driver.memory', '4g'),
 ('spark.rdd.compress', 'True'),
 ('spark.executor.memory', '4g'),
 ('spark.driver.port', '40021'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.executor.id', 'driver'),
 ('spark.submit.deployMode', 'client'),
 ('spark.app.id', 'local-1615496275942'),
 ('spark.app.name', 'word2vec'),
 ('spark.ui.showConsoleProgress', 'true')]

In [ ]:
# Look at vm os to get the number of cores
!cat /proc/cpuinfo

processor	: 0
vendor_id	: AuthenticAMD
cpu family	: 23
model		: 49
model name	: AMD EPYC 7B12
stepping	: 0
microcode	: 0x1000065
cpu MHz		: 2250.000
cache size	: 512 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid extd_apicid tsc_known_freq pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoext ssbd ibrs ibpb stibp vmmcall fsgsbase tsc_adjust bmi1 avx2 smep bmi2 rdseed adx smap clflushopt clwb sha_ni xsaveopt xsavec xgetbv1 clzero xsaveerptr arat npt nrip_save umip rdpid
bugs		: sysret_ss_attrs spectre_v1 spectre_v2 spec_store_bypass
bogomips	: 4500.00
TLB size	: 307

In [ ]:
# Look at vm os to get memory size
!cat /proc/meminfo

MemTotal:       13333568 kB
MemFree:          373764 kB
MemAvailable:   11164900 kB
Buffers:          115264 kB
Cached:         10581148 kB
SwapCached:            0 kB
Active:          4418788 kB
Inactive:        8105484 kB
Active(anon):    1701420 kB
Inactive(anon):      360 kB
Active(file):    2717368 kB
Inactive(file):  8105124 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               968 kB
Writeback:             0 kB
AnonPages:       1827760 kB
Mapped:           140948 kB
Shmem:              1124 kB
Slab:             336356 kB
SReclaimable:     294532 kB
SUnreclaim:        41824 kB
KernelStack:        5716 kB
PageTables:         9956 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666784 kB
Committed_AS:    5137308 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:             1024 kB
AnonHugePages:   

In [55]:
# Look for the spark java process to get memory config
!ps -ef | grep java

root        1185      59 35 20:57 ?        00:13:34 /usr/lib/jvm/java-8-openjdk-amd64/bin/java -cp /content/spark-2.4.7-bin-hadoop2.7/conf/:/content/spark-2.4.7-bin-hadoop2.7/jars/* -Xmx4g org.apache.spark.deploy.SparkSubmit --conf spark.executor.memory=4g --conf spark.driver.memory=4g --conf spark.app.name=word2vec pyspark-shell
root        2251      59  0 21:36 ?        00:00:00 /bin/bash -c ps -ef | grep java
root        2253    2251  0 21:36 ?        00:00:00 grep java
